$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
# Challenge 03: Minimum Makespan with Quantum Annealing

## Introduction

The core of this challenge is about translating a real-world problem into a mathematical representation that fits a quantum computer. In this challenge we encourage you to use two representations which even though equivalent, are used in different contexts. 

The first is Quadratic Unconstrained Binary Optimization (QUBO) which is used primarily for the quantum annealers (D-Wave) and the other is Ising Hamiltonian, which is often used with an algorithm called Quantum Approximate Optimization Algorithm (QAOA).

This challenge might require spending some time going through the references (especially 1-3), as we are not aware of any "quick introductions" worth mentioning. If you have a hard time understanding something, feel free to reach out for help on [QOSF Slack](https://join.slack.com/t/qosf/shared_invite/zt-bw59w8b9-WJ~k0~FAMHukTZov4AnLfA).

Unless you have a prior background with these kinds of problems, we encourage you to start from QUBO and then perhaps trying Ising Hamiltonians as well, as there are more materials about it.

It is also worth mentioning that D-Wave allows you to use their machines for free for a limited amount of time, so you can try running your QUBO on a real machine after signing up [here](https://cloud.dwavesys.com/leap/signup/).

## Problem

Niranjan owns a catering service and receives orders one day in advance. Each of the chefs he employs is paid by the hour. In order to optimize the cost of running his business, every night, he looks at all of the dishes he needs to deliver for the next day and asks his employees to work the specific hours he requires them for.

Given an order queue ($Q$) of length $N$ with each order $i$ taking $L_i$ time to prepare, if there are $m$ chefs, design a QUBO/Ising Hamiltonian to help Niranjan find the shortest time ($T$) it would take for the $m$ chefs to prepare all $N$ dishes. Assume each chef can only work on one dish at a time and each dish is worked on by only one chef. We can also safely assume that at any given point, all chefs can be actively working on a dish. 

Since Niranjan wants to ensure that he can run his algorithms on the near term devices, keep track and try to minimize the number of:
1. Gates & Qubits (for Ising Hamiltonian)
2. Variables (for QUBO. For embedding on a real machine also keep track of the lenghts of chains)

**Optional:** 
For the advanced reader, you can also attempt to build the Cost Hamiltonian or Mixer Hamiltonian as described in [7,8].

## Examples

### Case #1: 

*If,* $\quad N = 5 , \quad m = 1, \quad \text{queue} = \{2, 3, 7, 2, 1\} \quad$ *then,* $\quad T=15$

*since* there is only one chef, the shortest time it would take to complete all the orders is simply the sum of the times it would take for each individual order.

### Case #2:

*If,* $\quad N = 5 , \quad m = 2, \quad \text{queue} = \{2, 3, 7, 2, 1\} \quad$ *then,* $\quad T=8$

*since* there are two chefs $m_1$ and $m_2$, the distribution that yields the shortest time is:

$m_1 = \{2, 3, 2\}, \qquad m_2 = \{ 7, 1 \} $

### Case #3:

*If,* $\quad N = 5 , \quad m = 3, \quad \text{queue} = \{2, 3, 2, 2, 1\} \quad$ *then,* $\quad T=4$

*since* there are three chefs $m_1$, $m_2$ and $m_3$, the distribution that yields the shortest time is:

$m_1 = \{3 \}, \qquad m_2 = \{2,2 \}, \qquad m_3 = \{2,1 \} $ 

Note that there could be another possible distribution of tasks, but the shortest time does not change:

$m_1 = \{3,1 \} , \qquad m_2 = \{2,2 \} , \qquad m_3 = \{2 \} $

## Solution by peguerosdc

The third example in the previous section is what gave the hint and motivated this approach.

Defining $m_i$ as the queue of dishes assigned to chef i and defining the total cost of any queue as $cost(m_i)$, we first note that when a chef is assigned with the **optimal queue** $m_{optimal}$ (that is, the queue that satisfies $cost(m_{optimal})=T$), the rest of the chefs can be assigned any combination of the remaining dishes and they are all still valid solutions as long as they fall within the constraint that $cost(m_i) \leq T$ (because of the definition of $m_{optimal}$).

In both of the solutions of the previous example, the **optimal chef** (the one that is assigned with $m_{optimal}$) is always $m_2$, but that doesn't have to be the case as all the chefs are indistinguishable, so it really doesn't matter! That means we can pick any chef as the **optimal chef** (in the following, we will pick $m_1$) and try to minimize its cost subject the constraints that make it the true **optimal chef**:

1. Every dish can be prepared once and only once. This ensures that the solution found is a valid solution.
2. The time it takes to $m_1$ to cook all his dishes is enough time for the other chefs to also cook their dishes: 

$$cost(m_1) = T \geq cost(m_i) \quad \text{for any} \quad i\neq 1$$


To encode this problem into a QUBO/Ising optimisation problem, we use unary encoding to represent the dishes $d_i$ present in a given order Q of N dishes:

$$ \ket{d_0} = \ket{1000...0_N}$$

$$ \ket{d_1} = \ket{0100...0_N}$$

And so on. So, if the order is given by $Q=\{2,3,2,2,1\}$ and chef $m_1$ is preparing the first and the third dishes, his queue/state will be given by:

$$ \ket{m_1} = \ket{10100}$$

And the state $\ket{\Psi}$ of a system of $m$ chefs will be given by the tensor product (the $\otimes$ symbol is omitted for the sake of readability) of $m$ of these states:

$$ \ket{\Psi} = \ket{m_1}\ket{m_2}...\ket{m_m} $$

To find the most optimal solution to $\ket{\Psi}$, we need to build an Ising/QUBO Hamiltonian where the energy represents the cost $m_1$ (which is the quantity we want to minimize) subject to the 2 constraints mentioned above.

### Objective function and constraints

The QUBO Hamiltonian is coded in Qiskit and it is described in terms of the binary variables $\{ m_{ij} \}$ which will be 1 if the qubit j of the chef i is in the state $\ket{1}$ and 0 if it is in the state $\ket{0}$.

It is important to mention that even though Qiskit allows us to define the algorithm already in terms of $\{ m_{ij} \}$, we are describing each state in the basis of the $Z$ gate (that is, $\ket{1}$ represents a spin up state and $\ket{0}$ a spin down state), so if we wanted to run the algorithm on a real quantum machine (please someone correct me if I am wrong) or in some other simulators (like pyquil+Grove), we need to replace the variables $m_i$ with the following operators in terms of the $Z$ gate as their eigenvalues translate to the binary description we desire:

$$ m_i \rightarrow \hat{m_i} = \frac{1 - \hat{Z_i}}{2} \quad \text{because} \quad \hat{m_i}\ket{0_i} = 0, \quad \hat{m_i}\ket{1_i} = \ket{1_i}   $$

Following with the $\{ m_i \}$ description (again for the sake of readability), the cost of the optimal chef $m_1$ is encoded as:

$$ H_0 = \vec{Q} \cdot \vec{m_1} $$

Where $\vec{Q}$ is a vector of length $N$ where the entry $i$ corresponds to the cost of the dish $d_i$. Now, it is left to map the two constraints as energy penalties to ensure that the solution is going to be valid and optimal.

#### Constraint 1: every dish can be prepared once and only once

In terms of $\ket{\Psi}$, this constraint means that for any given dish j, only one $m_{ij}$ must be set for all posible values of i. This can be translated to an equality constraint with Lagrange multipliers:

$$ H_1 = \lambda_1 \sum_i^{N} \left( \sum_j^m m_{ji} -1 \right)^2 $$

#### Constraint 2: chef m1 must be the optimal chef

To consider this inequality constraint $cost(m_1) \geq cost(m_i)$ in our optimization problem, we need to translate it to an equality constraint with the aid of one slack variable:

$$ H_2 = \lambda_2 \sum_{i>1}^{m} \left( \vec{Q}\cdot (\vec{m_1} - \vec{m_i}) - s \right)^2 $$

This slack variable is bounded by the worst case scenario in which this constraint is violated, which is given by the case where no dishes are prepared by the optimal chef and all of them are prepared by any other chef $i$, so the value of $s$ is always gonna be capped by $cost(Q)$. Remembering that the algorithm is a **binary** algorithm, we need to (sort-of, but smarter) encode the slack variable in binary variables that will be represented by ancilla qubits.
All of this is already handled by Qiskit 👌

#### Hamiltonian

Putting everything together, the objective function is given by $H=H_0 + H_1 + H_2$:

$$ H = \vec{Q} \cdot \vec{m_1} + \lambda_1 \sum_i^{N} \left( \sum_j^m m_{ji} -1 \right)^2 + \lambda_2 \sum_{i>1}^{m} \left( \vec{Q}\cdot (\vec{m_1} - \vec{m_i}) - s \right)^2 $$

Which is trivially quantized to:

$$\hat{H} = \hat{\vec{Q}} \cdot \hat{\vec{m_1}} + \lambda_1 \sum_i^{N} \left( \sum_j^m \hat{m_{ji}} -1 \right)^2 + \lambda_2 \sum_{i>1}^{m} \left( \hat{\vec{Q}}\cdot (\hat{\vec{m_1}} - \hat{\vec{m_i}}) - \hat{s} \right)^2 $$

and its energy eigenvalues are given by:

$$E = cost(m_1) + \text{how much the constraints are violated}$$ 

Which means that the annealing process will (most likely) drive the system to the state where no constraints are violated and $cost(m_1)$ is at its minimum.

Finding the weights $\lambda_i$ of the penalties is far from a trivial task. Luckily, Qiskit is handling this for us as well.

### Coding the solution

In [1]:
# Importing optimization Qiskit library
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization.converters import InequalityToEquality, IntegerToBinary, LinearEqualityToPenalty
# Importing standard Qiskit libraries
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA

class MinimumMakespanSolver(object):

    """docstring for MinimumMakespanSolver"""
    def __init__(self, queue, chefs):
        self.queue = queue
        self.N = len(queue)
        self.chefs = chefs
        self.raw_program = self.build()
        self.program = self.raw_to_qubo()

    def dish_to_qubit(self, dish, chef):
        return chef*self.N + dish

    def get_qubits_of_chef(self, chef):
        return [ f"x{self.dish_to_qubit(dish,chef)}" for dish in range(self.N) ] 

    def build(self):
        # Define the program with enough qubits to encode the states as:
        # |queue1>|queue2>...|queue_m>
        qp = QuadraticProgram()
        for b in range(self.N*self.chefs):
            qp.binary_var(f"x{b}")
            
        # The cost of the first/optimal chef is the objective function
        # to minimize
        qp.minimize(linear=self.queue)
        # Constraint 1: every dish must to be prepared once and only once
        for dish in range(self.N):
            constraint = dict()
            for chef in range(self.chefs):
                var = f"x{self.dish_to_qubit(dish,chef)}"
                constraint[var] = 1
            qp.linear_constraint(linear=constraint, sense='=', rhs=1, name=f"simultaneous{dish}")
        # Constraint 2: chef 1 must be the optimal chef
        dishes_optimal = self.get_qubits_of_chef(0)
        for chef in range(1,self.chefs):
            dishes_this = self.get_qubits_of_chef(chef)
            cost = dict()
            for i,dish in enumerate(queue):
                cost[dishes_optimal[i]] = dish
                cost[dishes_this[i]] = -dish
            qp.linear_constraint(linear=cost, sense='>=', rhs=0,name=f"max_chef{chef}")
        return qp

    def raw_to_qubo(self):
        # First, translate the inequality constraints to equality constraints
        qp_eq = InequalityToEquality().convert(self.raw_program)
        # Then, translate the integers in the constraints as binary variables (this is
        # where ancilla qubits are added)
        qp_eq_bin = IntegerToBinary().convert(qp_eq)
        # And then we put everything together and the constraints are mapped to penalties
        # in terms of Lagrange multipliers
        qubo = LinearEqualityToPenalty().convert(qp_eq_bin)
        return qubo

    def solve(self, shots=1000):
        # Simulate optimization
        quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),shots=shots)
        qaoa_mes = QAOA(quantum_instance=quantum_instance)
        qaoa = MinimumEigenOptimizer(qaoa_mes) 
        return qaoa.solve(self.program)

    def raw_summary(self):
        return self.raw_program.export_as_lp_string()

    def summary(self):
        return self.program.export_as_lp_string()

And here are some useful functions to show the result:

In [2]:
def state_to_dishes(state, queue, m):
    N = len(queue)
    result = []
    # Get the state of every chef
    for chef in range(m):
        this_state = [dish if state[chef*N + i]=='1' else 0 for i, dish in enumerate(queue)]
        result.append(this_state)
    return result

def stringify_state(state, queue, m):
    dishes = state_to_dishes(state, queue, m)
    return [f"Chef {chef} cooked {result} with cost {sum(result)}" for result,chef in zip(dishes,range(m))]

def visualize_solution(solution, queue, m):
    base_cost = solution.samples[0][1]
    for state, cost, prob in solution.samples:
        # if the cost is different, this solution is no longer optimal
        if cost != base_cost:
            break
        # print the state
        for s in stringify_state(state, queue, m):
            print(s)
        print("-------")

#### Case 1

In [3]:
queue = [2,3,7,2,1]
m = 1
solver = MinimumMakespanSolver(queue=queue, chefs=m)
print(solver.raw_summary())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 2 x0 + 3 x1 + 7 x2 + 2 x3 + x4
Subject To
 simultaneous0: x0 = 1
 simultaneous1: x1 = 1
 simultaneous2: x2 = 1
 simultaneous3: x3 = 1
 simultaneous4: x4 = 1

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x0 x1 x2 x3 x4
End



In [4]:
solution = solver.solve()
visualize_solution(solution, queue, m)

Chef 0 cooked [2, 3, 7, 2, 1] with cost 15
-------


#### Case 2

In [5]:
queue = [2,3,7,2,1]
m = 2
solver = MinimumMakespanSolver(queue=queue, chefs=m)
print(solver.raw_summary())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 2 x0 + 3 x1 + 7 x2 + 2 x3 + x4
Subject To
 simultaneous0: x0 + x5 = 1
 simultaneous1: x1 + x6 = 1
 simultaneous2: x2 + x7 = 1
 simultaneous3: x3 + x8 = 1
 simultaneous4: x4 + x9 = 1
 max_chef1: 2 x0 + 3 x1 + 7 x2 + 2 x3 + x4 - 2 x5 - 3 x6 - 7 x7 - 2 x8 - x9 >= 
            0

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1

Binaries
 x0 x1 x2 x3 x4 x5 x6 x7 x8 x9
End



In [6]:
solution = solver.solve(50000)
visualize_solution(solution, queue, m)

Chef 0 cooked [2, 3, 0, 2, 1] with cost 8
Chef 1 cooked [0, 0, 7, 0, 0] with cost 7
-------
Chef 0 cooked [0, 0, 7, 0, 1] with cost 8
Chef 1 cooked [2, 3, 0, 2, 0] with cost 7
-------


#### Case 3

In [7]:
queue = [2,3,2,2,1]
m = 3
solver = MinimumMakespanSolver(queue=queue, chefs=m)
print(solver.summary())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 20 x0 - 19 x1 - 20 x2 - 20 x3 - 21 x4 - 22 x5 - 22 x6 - 22 x7 - 22 x8
      - 22 x9 - 22 x10 - 22 x11 - 22 x12 - 22 x13 - 22 x14 + [ 198 x0^2
      + 528 x0*x1 + 352 x0*x2 + 352 x0*x3 + 176 x0*x4 - 132 x0*x5 - 264 x0*x6
      - 176 x0*x7 - 176 x0*x8 - 88 x0*x9 - 132 x0*x10 - 264 x0*x11 - 176 x0*x12
      - 176 x0*x13 - 88 x0*x14 - 88 x0*max_chef1@int_slack@0
      - 176 x0*max_chef1@int_slack@1 - 352 x0*max_chef1@int_slack@2
      - 264 x0*max_chef1@int_slack@3 - 88 x0*max_chef2@int_slack@0
      - 176 x0*max_chef2@int_slack@1 - 352 x0*max_chef2@int_slack@2
      - 264 x0*max_chef2@int_slack@3 + 418 x1^2 + 528 x1*x2 + 528 x1*x3
      + 264 x1*x4 - 264 x1*x5 - 352 x1*x6 - 264 x1*x7 - 264 x1*x8 - 132 x1*x9
      - 264 x1*x10 - 352 x1*x11 - 264 x1*x12 - 264 x1*x13 - 132 x1*x14
      - 132 x1*max_chef1@int_slack@0 - 264 x1*max_chef1@int_slack@1
      - 528 x1*max_chef1@int_slack@2 - 396 x

In [8]:
solution = solver.solve(50000)
visualize_solution(solution, queue, m)

Chef 0 cooked [2, 0, 2, 0, 0] with cost 4
Chef 1 cooked [0, 3, 0, 0, 1] with cost 4
Chef 2 cooked [0, 0, 0, 2, 0] with cost 2
-------


## Resources

\[1\] [D-Wave Problem Solving Handbook](https://docs.dwavesys.com/docs/latest/doc_handbook.html)

\[2\] [Implementation of the Travelling Salesman Problem](https://github.com/mstechly/quantum_tsp_tutorials)

\[3\] [Ising formulations of many NP problems](https://arxiv.org/pdf/1302.5843.pdf)

\[4\] [D-Wave Examples](https://docs.ocean.dwavesys.com/en/stable/getting_started.html#examples)

\[5\] [Quantum Approximate Optimization Algorithm explained](https://www.mustythoughts.com/quantum-approximate-optimization-algorithm-explained)

\[6\] [Job Shop Scheduling Solver based on Quantum Annealing](https://arxiv.org/pdf/1506.08479.pdf)

### References

\[7\] [Quantum Algorithms for Scientific Computing and Approximate Optimization](https://arxiv.org/pdf/1805.03265.pdf)

\[8\] [From the Quantum Approximate Optimization Algorithm to a Quantum Alternating Operator Ansatz](https://arxiv.org/pdf/1709.03489.pdf)

### Other useful resources
\[9\] [D-Wave Getting Started with the System](https://docs.dwavesys.com/docs/latest/doc_getting_started.html)

\[10\] [Qiskit Converters for Quadratic Programs](https://qiskit.org/documentation/tutorials/optimization/2_converters_for_quadratic_programs.html)

\[11\] [D-Wave Support Difference between BQM, Ising, and QUBO problems?](https://support.dwavesys.com/hc/en-us/community/posts/360017439853-Difference-between-BQM-Ising-and-QUBO-problems-)

\[12\] [A QAOA solution to the traveling salesman problem using pyQuil](https://cs269q.stanford.edu/projects2019/radzihovsky_murphy_swofford_Y.pdf)